In [ ]:
%%capture

!pip install transformers langchain accelerate wandb bitsandbytes
!pip install datasets
!pip install -U flash-attn --no-build-isolation

In [ ]:
import os
import getpass
import wandb
import torch
import numpy as np
import pandas as pd
from datasets import Dataset, load_dataset

from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, BitsAndBytesConfig, GenerationConfig

In [ ]:
!git clone https://github.com/harpreetsahota204/text-gen-utils.git

In [ ]:
import sys
sys.path.append('/content/text-gen-utils/text-gen-utils')

In [ ]:
from text_gen_utils import gen_pipeline, run_single_param_experiment, run_experiments, instantiate_huggingface_model

In [ ]:
torch.manual_seed(42)

In [ ]:
!huggingface-cli login

In [ ]:
!huggingface-cli whoami

In [ ]:
os.environ["WANDB_API_KEY"] = getpass.getpass("Enter your WandB API key: ")

In [ ]:
red_teaming_prompts = load_dataset('harpreetsahota/elicit-offensive-language-prompts', split='train')

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
gen_params = {
    "temperature": [1e-3, .25, .50, .75, 1.0],
    "num_beams": [1, 3, 5, 7],
}

In [ ]:
decilm = AutoModelForCausalLM.from_pretrained("Deci/DeciLM-7b-redteam4-lm_eval",
                                              device_map="auto",
                                              quantization_config=bnb_config,
                                              trust_remote_code=True)

mistral = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1",
                                               device_map="auto",
                                               quantization_config=bnb_config,
                                               trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1",
                                          padding_side="left")

tokenizer.pad_token_id = tokenizer.eos_token_id

deci_generator = pipeline("text-generation",
                          model=decilm,
                          tokenizer=tokenizer,
                          device_map="auto",
                          do_sample=True,
                          early_stopping=True,
                          max_length=250
)

mistral_generator = pipeline("text-generation",
                          model=mistral,
                          tokenizer=tokenizer,
                          device_map="auto",
                          do_sample=True,
                          early_stopping=True,
                          max_length=250,
)

pipelines = {
    'DeciLM': deci_generator,
    'Mistral': mistral_generator
}

In [ ]:
results = run_experiments(pipelines,
                          red_teaming_prompts,
                          gen_params,
                          gen_pipeline,
                          'Prompt',
                          'red-teaming-offensive-language',
                          'harp-deci')

In [ ]:
import pandas as pd
results_df = pd.DataFrame(results)

generated_dataset = Dataset.from_dict(results_df)

generated_dataset.push_to_hub('harpreetsahota/red-team-offensive-language-results', private=True)